# 🚗 Drowsy Driver Detection System

---

## 📋 Informasi Proyek

**Judul**: Drowsy Driver Detection System menggunakan Vision Transformer (ViT)

**Deskripsi**: Sistem deteksi kantuk pengemudi secara real-time menggunakan Computer Vision dan Deep Learning

---

## 🎯 Topik Computer Vision yang Tercakup

1. **Object Detection** - Deteksi wajah menggunakan Haar Cascade
2. **Object Tracking** - Tracking wajah frame-by-frame
3. **Object Recognition** - Klasifikasi drowsy/not drowsy
4. **CNN (Vision Transformer)** - State-of-the-art deep learning architecture

---

## 🤖 Informasi Model

- **Architecture**: Vision Transformer (ViT-Base)
- **Parameters**: 86M parameters
- **Accuracy**: 97.52% 
- **Dataset**: UTA-RLDD (Real-Life Drowsiness Dataset)
- **Classes**: 
  - 0: Not Drowsy
  - 1: Drowsy

---

## 📦 1. Import Libraries

Import semua library yang dibutuhkan untuk project ini

In [ ]:
# Computer Vision
import cv2  # OpenCV untuk video processing
from PIL import Image  # PIL untuk image manipulation

# Deep Learning
import torch  # PyTorch framework
from transformers import ViTForImageClassification, ViTImageProcessor  # Hugging Face Transformers

# Data processing
import numpy as np  # Numerical operations
import pandas as pd  # Data analysis (untuk log)

# Utilities
import time  # Timing operations
import os  # File operations
import sys  # System operations
from datetime import datetime  # Timestamp

# Visualization
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Statistical visualization

# Alert system
import pygame  # Sound playback

# Jupyter-specific
from IPython.display import display, clear_output

print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"OpenCV version: {cv2.__version__}")

## 🔧 2. Setup Configuration

Konfigurasi path dan parameters untuk sistem

In [ ]:
# ==================== PATHS ====================
MODEL_PATH = "./models"  # Path ke folder model ViT
ALERT_SOUND_PATH = "./assets/alert.wav"  # Path ke alert sound
LOG_PATH = "./data/drowsy_log.csv"  # Path untuk save log
OUTPUT_VIDEO_PATH = None  # Set path jika mau save video output

# ==================== PARAMETERS ====================
DROWSY_THRESHOLD = 15  # Alert jika drowsy 15 frames berturut-turut (~0.5 detik at 30 FPS)
PREDICTION_INTERVAL = 3  # Predict setiap 3 frame (untuk performa)
VIDEO_SOURCE = 0  # 0 untuk webcam, atau path video file

# ==================== DEVICE ====================
# Gunakan GPU jika tersedia, kalau tidak pakai CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📱 Device: {device}")

# ==================== VERIFICATION ====================
# Cek apakah model folder exists
if not os.path.exists(MODEL_PATH):
    print(f"❌ Error: Model folder not found at {MODEL_PATH}")
else:
    print(f"✅ Model folder found at {MODEL_PATH}")
    # List files in model folder
    model_files = os.listdir(MODEL_PATH)
    print(f"   Files: {model_files}")

# Create data folder if not exists
os.makedirs("./data", exist_ok=True)
print("✅ Configuration setup complete!")

## 🤖 3. Load Vision Transformer Model

Load pre-trained ViT model untuk klasifikasi drowsiness

In [ ]:
print("🤖 Loading Vision Transformer model...\n")

try:
    # Load image processor (untuk preprocessing)
    processor = ViTImageProcessor.from_pretrained(MODEL_PATH)
    print("✅ ViT Processor loaded")
    
    # Load model
    model = ViTForImageClassification.from_pretrained(MODEL_PATH)
    model.to(device)  # Pindah ke GPU/CPU
    model.eval()  # Set ke evaluation mode (tidak training)
    print("✅ ViT Model loaded")
    
    # Print model info
    print("\n📊 Model Information:")
    print(f"   Model type: {model.config.model_type}")
    print(f"   Image size: {model.config.image_size}x{model.config.image_size}")
    print(f"   Number of classes: {len(model.config.id2label)}")
    print(f"   Classes: {model.config.id2label}")
    print(f"   Hidden size: {model.config.hidden_size}")
    print(f"   Num layers: {model.config.num_hidden_layers}")
    print(f"   Num attention heads: {model.config.num_attention_heads}")
    
    # Calculate total parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\n   Total parameters: {total_params:,}")
    print(f"   Trainable parameters: {trainable_params:,}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise

## 👁️ 4. Setup Face Detection

Setup Haar Cascade classifier untuk deteksi wajah (Object Detection)

In [ ]:
print("👁️ Setting up Face Detection...\n")

# Load Haar Cascade classifier untuk face detection
# Haar Cascade adalah metode classical computer vision (bukan deep learning)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

# Verifikasi cascade loaded
if face_cascade.empty():
    print("❌ Error: Haar Cascade not loaded!")
else:
    print("✅ Haar Cascade Face Detector loaded successfully!")
    print("   Method: Viola-Jones Algorithm (2001)")
    print("   Type: Classical Computer Vision (non-ML)")

def detect_face(frame):
    """
    Deteksi wajah dari frame menggunakan Haar Cascade
    
    Args:
        frame: Input frame (BGR format)
        
    Returns:
        face_img: Cropped face image
        coords: Tuple (x, y, w, h) koordinat face
    """
    # Convert ke grayscale (Haar Cascade bekerja pada grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    # scaleFactor: parameter untuk image pyramid (1.3 = reduce 30% per level)
    # minNeighbors: minimum neighbors untuk valid detection (higher = more strict)
    # minSize: minimum face size in pixels
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(100, 100)
    )
    
    if len(faces) > 0:
        # Ambil face pertama (atau face terbesar jika ada multiple)
        (x, y, w, h) = faces[0]
        face_img = frame[y:y+h, x:x+w]
        return face_img, (x, y, w, h)
    
    return None, None

print("\n✅ Face detection function defined!")

## 🎨 5. Image Preprocessing Function

Fungsi untuk preprocess face image sebelum input ke model ViT

In [ ]:
def preprocess_face(face_img):
    """
    Preprocess face image untuk ViT model
    
    Steps:
    1. Convert BGR (OpenCV) ke RGB (PIL/Model)
    2. Convert numpy array ke PIL Image
    3. Resize ke 224x224 (ViT input size)
    4. Normalize pixel values
    5. Convert ke tensor
    
    Args:
        face_img: Face image (BGR format dari OpenCV)
        
    Returns:
        inputs: Preprocessed tensor siap untuk model
    """
    # Convert BGR ke RGB
    face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    
    # Convert numpy array ke PIL Image
    pil_image = Image.fromarray(face_rgb)
    
    # Preprocess menggunakan ViT processor
    # Processor akan: resize, normalize, convert to tensor
    inputs = processor(images=pil_image, return_tensors="pt")
    
    return inputs

print("✅ Preprocessing function defined!")
print("   Input: BGR image from OpenCV")
print("   Output: Preprocessed tensor (1, 3, 224, 224)")

## 🔮 6. Prediction Function

Fungsi untuk predict drowsiness dari face image (Object Recognition)

In [ ]:
def predict_drowsiness(face_img):
    """
    Predict drowsiness dari face image menggunakan ViT model
    
    Args:
        face_img: Face image (BGR format)
        
    Returns:
        label: 'drowsy' atau 'notdrowsy'
        confidence: Confidence score (0-1)
    """
    try:
        # Preprocess image
        inputs = preprocess_face(face_img)
        
        # Move inputs ke device (GPU/CPU)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Inference (forward pass)
        with torch.no_grad():  # Tidak perlu gradient (tidak training)
            outputs = model(**inputs)
            logits = outputs.logits  # Raw scores dari model
        
        # Get predicted class (argmax)
        predicted_class = logits.argmax(-1).item()
        
        # Calculate confidence scores menggunakan softmax
        probabilities = torch.nn.functional.softmax(logits, dim=-1)[0]
        confidence = probabilities[predicted_class].item()
        
        # Get label dari id2label mapping
        label = model.config.id2label[str(predicted_class)]
        
        return label, confidence
        
    except Exception as e:
        print(f"⚠️ Prediction error: {e}")
        return None, 0.0

print("✅ Prediction function defined!")
print("   Input: Face image (H, W, 3)")
print("   Output: Label (drowsy/notdrowsy) + Confidence (0-1)")

## 🔊 7. Alert System

Setup sistem alert (visual dan audio) ketika drowsiness terdeteksi

In [ ]:
# Initialize pygame mixer untuk audio
pygame.mixer.init()

# Load alert sound jika file exists
alert_sound = None
if os.path.exists(ALERT_SOUND_PATH):
    try:
        alert_sound = pygame.mixer.Sound(ALERT_SOUND_PATH)
        print(f"✅ Alert sound loaded from {ALERT_SOUND_PATH}")
    except Exception as e:
        print(f"⚠️ Could not load alert sound: {e}")
else:
    print(f"⚠️ Alert sound not found at {ALERT_SOUND_PATH}")
    print("   Visual alert will still work")

def trigger_visual_alert(frame, text="⚠️ DROWSINESS DETECTED! ⚠️"):
    """
    Tampilkan visual alert di frame
    
    Args:
        frame: Video frame
        text: Alert text
        
    Returns:
        frame: Frame dengan alert overlay
    """
    # Overlay merah semi-transparent di bagian atas
    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (frame.shape[1], 100), (0, 0, 255), -1)
    frame = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)
    
    # Text alert
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)[0]
    text_x = (frame.shape[1] - text_size[0]) // 2
    cv2.putText(frame, text, (text_x, 60),
               cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
    
    return frame

def trigger_audio_alert():
    """Play alert sound jika tersedia"""
    if alert_sound:
        alert_sound.play()

print("\n✅ Alert system setup complete!")

## 📊 8. Visualization Function

Fungsi untuk visualisasi informasi di frame

In [ ]:
def draw_info(frame, face_coords, label, confidence, fps, stats):
    """
    Gambar informasi lengkap di frame
    
    Args:
        frame: Video frame
        face_coords: Koordinat face (x, y, w, h)
        label: Prediction label
        confidence: Confidence score
        fps: Frame per second
        stats: Dictionary dengan statistik (total_frames, drowsy_count, etc)
        
    Returns:
        frame: Frame dengan overlay informasi
    """
    height, width = frame.shape[:2]
    
    # ========== FACE BOUNDING BOX ==========
    if face_coords is not None:
        (x, y, w, h) = face_coords
        
        # Warna: merah jika drowsy, hijau jika awake
        color = (0, 0, 255) if label == "drowsy" else (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        
        # Label di atas bounding box
        label_text = f"{label.upper()}: {confidence:.1%}"
        cv2.putText(frame, label_text, (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    
    # ========== INFO PANEL (Kiri Atas) ==========
    info_y = 30
    cv2.putText(frame, f"FPS: {fps:.1f}", (10, info_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    info_y += 30
    cv2.putText(frame, f"Frames: {stats['total_frames']}", (10, info_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    info_y += 25
    cv2.putText(frame, f"Drowsy: {stats['drowsy_count']}", (10, info_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    info_y += 25
    cv2.putText(frame, f"Alerts: {stats['alert_count']}", (10, info_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # ========== STATUS (Kanan Atas) ==========
    status_text = "STATUS: DROWSY!" if label == "drowsy" else "STATUS: AWAKE"
    status_color = (0, 0, 255) if label == "drowsy" else (0, 255, 0)
    text_size = cv2.getTextSize(status_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
    status_x = width - text_size[0] - 10
    cv2.putText(frame, status_text, (status_x, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, status_color, 2)
    
    # ========== DROWSINESS METER ==========
    if label == "drowsy":
        bar_width = 200
        bar_height = 20
        bar_x = width - bar_width - 10
        bar_y = 50
        
        # Background
        cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height),
                     (100, 100, 100), -1)
        
        # Progress
        progress = min(stats['drowsy_counter'] / DROWSY_THRESHOLD, 1.0)
        progress_width = int(bar_width * progress)
        cv2.rectangle(frame, (bar_x, bar_y), (bar_x + progress_width, bar_y + bar_height),
                     (0, 0, 255), -1)
        
        # Text
        cv2.putText(frame, f"Alert in: {max(0, DROWSY_THRESHOLD - stats['drowsy_counter'])}", 
                   (bar_x, bar_y - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    return frame

print("✅ Visualization function defined!")

## 🎥 9. Main Detection Loop

Real-time drowsiness detection dari webcam/video

**Controls:**
- Press **'q'** to quit
- Press **'r'** to reset statistics

In [ ]:
print("="*70)
print("STARTING DROWSY DRIVER DETECTION")
print("="*70)
print("\nPress 'q' to quit, 'r' to reset statistics\n")

# ==================== INITIALIZATION ====================
# Open video capture
cap = cv2.VideoCapture(VIDEO_SOURCE)

if not cap.isOpened():
    print("❌ Error: Cannot open camera/video!")
else:
    print("✅ Video capture started!")
    
    # Video writer setup (jika mau save output)
    video_writer = None
    if OUTPUT_VIDEO_PATH:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps_out = int(cap.get(cv2.CAP_PROP_FPS))
        width_out = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height_out = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        video_writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps_out, (width_out, height_out))
        print(f"📹 Saving output to: {OUTPUT_VIDEO_PATH}")
    
    # Log file setup
    log_file = open(LOG_PATH, "w")
    log_file.write("timestamp,frame,label,confidence,drowsy_counter,alert\n")
    print(f"📝 Saving log to: {LOG_PATH}")
    
    # Statistics
    stats = {
        'total_frames': 0,
        'drowsy_count': 0,
        'alert_count': 0,
        'drowsy_counter': 0
    }
    
    # FPS calculation
    fps_start_time = time.time()
    fps_frame_count = 0
    fps = 0
    
    # Prediction caching
    frame_count = 0
    last_prediction = None
    last_confidence = 0.0
    
    print("\n▶️  Detection started!\n")
    
    # ==================== MAIN LOOP ====================
    try:
        while True:
            # Capture frame
            ret, frame = cap.read()
            if not ret:
                print("⚠️ End of video or cannot read frame")
                break
            
            stats['total_frames'] += 1
            frame_count += 1
            fps_frame_count += 1
            
            # Calculate FPS
            if time.time() - fps_start_time >= 1.0:
                fps = fps_frame_count / (time.time() - fps_start_time)
                fps_start_time = time.time()
                fps_frame_count = 0
            
            # ========== FACE DETECTION ==========
            face_img, face_coords = detect_face(frame)
            
            if face_img is not None:
                # ========== DROWSINESS PREDICTION ==========
                # Predict setiap PREDICTION_INTERVAL frames (untuk performa)
                if frame_count % PREDICTION_INTERVAL == 0:
                    label, confidence = predict_drowsiness(face_img)
                    if label is not None:
                        last_prediction = label
                        last_confidence = confidence
                else:
                    # Gunakan prediksi terakhir
                    label = last_prediction
                    confidence = last_confidence
                
                # ========== ALERT LOGIC ==========
                if label == "drowsy":
                    stats['drowsy_counter'] += 1
                    stats['drowsy_count'] += 1
                    
                    # Trigger alert jika melewati threshold
                    if stats['drowsy_counter'] >= DROWSY_THRESHOLD:
                        # Visual alert
                        frame = trigger_visual_alert(frame)
                        
                        # Audio alert
                        trigger_audio_alert()
                        
                        # Log alert
                        if stats['drowsy_counter'] == DROWSY_THRESHOLD:
                            stats['alert_count'] += 1
                            print(f"🚨 ALERT #{stats['alert_count']} at frame {stats['total_frames']}")
                else:
                    # Reset counter jika awake
                    stats['drowsy_counter'] = 0
                
                # ========== VISUALIZATION ==========
                frame = draw_info(frame, face_coords, label, confidence, fps, stats)
                
                # ========== LOGGING ==========
                alert_status = "YES" if stats['drowsy_counter'] >= DROWSY_THRESHOLD else "NO"
                log_file.write(f"{time.time()},{stats['total_frames']},{label},{confidence:.4f},{stats['drowsy_counter']},{alert_status}\n")
            
            else:
                # No face detected
                cv2.putText(frame, "No face detected", (50, 50),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                stats['drowsy_counter'] = 0
            
            # ========== OUTPUT ==========
            if video_writer:
                video_writer.write(frame)
            
            # Show frame
            cv2.imshow('Drowsy Driver Detection', frame)
            
            # ========== KEYBOARD CONTROLS ==========
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("\n⏹️  Stopping detection...")
                break
            elif key == ord('r'):
                print("\n🔄 Resetting statistics...")
                stats = {
                    'total_frames': 0,
                    'drowsy_count': 0,
                    'alert_count': 0,
                    'drowsy_counter': 0
                }
    
    except KeyboardInterrupt:
        print("\n⏹️  Interrupted by user")
    
    finally:
        # ==================== CLEANUP ====================
        print("\n🧹 Cleaning up...")
        cap.release()
        if video_writer:
            video_writer.release()
        log_file.close()
        cv2.destroyAllWindows()
        
        # ==================== SUMMARY ====================
        print("\n" + "="*70)
        print("DETECTION SUMMARY")
        print("="*70)
        print(f"Total Frames: {stats['total_frames']}")
        print(f"Drowsy Detections: {stats['drowsy_count']}")
        print(f"Alerts Triggered: {stats['alert_count']}")
        if stats['total_frames'] > 0:
            drowsy_rate = (stats['drowsy_count'] / stats['total_frames']) * 100
            print(f"Drowsiness Rate: {drowsy_rate:.2f}%")
        print("="*70)
        print("✅ Detection completed!")

## 📊 10. Analyze Detection Log

Analisis hasil deteksi dari log CSV

In [ ]:
# Load log data
if os.path.exists(LOG_PATH):
    df = pd.read_csv(LOG_PATH)
    
    print("📊 Log Data Analysis\n")
    print(f"Total records: {len(df)}")
    print(f"\nLabel distribution:")
    print(df['label'].value_counts())
    
    print(f"\nAlert statistics:")
    print(df['alert'].value_counts())
    
    print(f"\nConfidence statistics:")
    print(df['confidence'].describe())
    
    # Display first few rows
    print(f"\nFirst 10 records:")
    display(df.head(10))
else:
    print(f"⚠️ Log file not found at {LOG_PATH}")

## 📈 11. Visualize Results

Visualisasi hasil deteksi menggunakan matplotlib

In [ ]:
if os.path.exists(LOG_PATH) and len(df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Label distribution
    df['label'].value_counts().plot(kind='bar', ax=axes[0, 0], color=['green', 'red'])
    axes[0, 0].set_title('Detection Distribution', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Label')
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].tick_params(axis='x', rotation=0)
    
    # Plot 2: Confidence over time
    axes[0, 1].plot(df['frame'], df['confidence'], alpha=0.7)
    axes[0, 1].set_title('Confidence Score Over Time', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Frame')
    axes[0, 1].set_ylabel('Confidence')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Drowsy counter over time
    axes[1, 0].plot(df['frame'], df['drowsy_counter'], color='red', alpha=0.7)
    axes[1, 0].axhline(y=DROWSY_THRESHOLD, color='orange', linestyle='--', label=f'Alert Threshold ({DROWSY_THRESHOLD})')
    axes[1, 0].set_title('Drowsiness Counter Over Time', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Frame')
    axes[1, 0].set_ylabel('Counter')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Alert distribution
    df['alert'].value_counts().plot(kind='pie', ax=axes[1, 1], autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
    axes[1, 1].set_title('Alert Distribution', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('')
    
    plt.tight_layout()
    plt.savefig('data/detection_analysis.png', dpi=300, bbox_inches='tight')
    print("✅ Visualization saved to data/detection_analysis.png")
    plt.show()
else:
    print("⚠️ No data to visualize")

## 🎓 12. Test with Single Image

Test model dengan single image (untuk debugging)

In [ ]:
# Test dengan capture frame dari webcam
cap_test = cv2.VideoCapture(0)
ret, test_frame = cap_test.read()
cap_test.release()

if ret:
    # Detect face
    face_img, face_coords = detect_face(test_frame)
    
    if face_img is not None:
        # Predict
        label, confidence = predict_drowsiness(face_img)
        
        # Draw result
        (x, y, w, h) = face_coords
        color = (0, 0, 255) if label == "drowsy" else (0, 255, 0)
        cv2.rectangle(test_frame, (x, y), (x+w, y+h), color, 3)
        cv2.putText(test_frame, f"{label}: {confidence:.2%}", (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        # Display
        plt.figure(figsize=(10, 8))
        plt.imshow(cv2.cvtColor(test_frame, cv2.COLOR_BGR2RGB))
        plt.title(f"Prediction: {label.upper()} ({confidence:.1%})", fontsize=16)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        print(f"\n✅ Test completed!")
        print(f"   Prediction: {label}")
        print(f"   Confidence: {confidence:.2%}")
    else:
        print("❌ No face detected in test image")
else:
    print("❌ Cannot capture test image")

## 📝 13. Summary & Conclusion

### 🎯 Topik Computer Vision yang Diimplementasikan:

1. **Object Detection** ✅
   - Haar Cascade untuk face detection
   - Classical computer vision approach (Viola-Jones)

2. **Object Tracking** ✅
   - Frame-by-frame face tracking
   - Drowsiness counter tracking

3. **Object Recognition** ✅
   - Binary classification (drowsy/not drowsy)
   - Vision Transformer (ViT) architecture
   - 97.52% accuracy

4. **CNN/Transformers** ✅
   - ViT-Base (86M parameters)
   - State-of-the-art deep learning
   - Self-attention mechanism

### 📊 Performance:

- **Model Accuracy**: 97.52%
- **Real-time FPS**: 20-30 FPS (depending on hardware)
- **Alert Response Time**: ~0.5 seconds (15 frames at 30 FPS)

### 🚀 Future Improvements:

1. Add Eye Aspect Ratio (EAR) calculation
2. Implement head pose estimation
3. Add yawn detection
4. Multi-face support
5. Cloud deployment

---

**Author**: [Your Name]

**Date**: December 2024

**Course**: Computer Vision - Final Project

---